In [1]:
import pickle 
import numpy as np
import os

from fairsoft_utils import formal_fairness_name, formal_metric_name, formal_model_name

In [2]:
def shorten_se(se):
    if se == 0:
        return '0'
    else:
        return f'\\nicefrac{{ {int(se * 1000)} }}{{10^3}}'

In [3]:
def show_valid_replication(dataset, reg_norm, target_label_idx=0, masked=False, fair_coeff=1):
    
    fair_metrics = []
    fair_results = {}
    # prefix = f'inprocess/evaluation-{dataset}/evaluation-{target_label_idx}'
    prefix = f'new_fair_through_distance/model/{dataset}/evaluation-{target_label_idx}'

    if masked: 
        prefix += '_masked'
    for i in range(1, 11):
        target_file = f'{prefix}/fair_eval_{reg_norm}_reg_lambda={fair_coeff:.2f}_{i:04d}.pkl'
        if os.path.exists(target_file):
            fairs = pickle.load(open(target_file, 'rb'))
            # print(fairs)
            if not fair_metrics:
                fair_metrics = list(fairs.keys())
            # print(fair_metrics)
            for met in fairs:
                if met not in fair_results:
                    fair_results[met] = {}
                # print(met, fairs[met].keys())
                for mod in fairs[met]:
                    if mod not in fair_results[met]:
                        fair_results[met][mod] = []
                    fair_results[met][mod].append(fairs[met][mod])

    perform_metrics = []
    perform_results = {}
    for i in range(1, 11):
        target_file = f'{prefix}/perform_eval_{reg_norm}_reg_lambda={fair_coeff:.2f}_{i:04d}.pkl'
        if os.path.exists(target_file):
            performs = pickle.load(open(target_file, 'rb'))
            
            perform_models = list(performs.keys())
            
            if not perform_metrics:
                perform_metrics = list(performs[perform_models[0]].keys())
            
            for met in perform_metrics:
                if met not in perform_results:
                    perform_results[met] =  {}
                for mod in performs:
                    if mod not in perform_results[met]:
                        perform_results[met][mod] = []
                    
                    perform_results[met][mod].append(performs[mod][met])
            
    fair_metrics = list(fair_results.keys())
    fair_metrics_nested = {}
    fair_metrics_sorted = []
    should_add_eo = False
    for met_hparam in fair_metrics:
        met = met_hparam.split('_')[0]
        if met not in fair_metrics_nested:
            fair_metrics_nested[met] = []
        fair_metrics_nested[met].append(met_hparam)
    
    for met in ['constant', 'jaccard', 'indication', 'elementwise']:
        if met in fair_metrics_nested:
            if len(fair_metrics_nested[met]) > 1:
                met_sorted = sorted(
                    fair_metrics_nested[met], key=lambda met: float(met.split('_')[-1]))
            else:
                met_sorted = fair_metrics_nested[met]
            fair_metrics_sorted += met_sorted

    fair_metrics = fair_metrics_sorted
    fair_models = [formal_model_name(fair_metric).replace('\\', '') for fair_metric in fair_metrics]

    colnames = ' & ' + ' & '.join(fair_models + ['w/o Reg'])
    print(colnames + '\\\\')
    print('\\midrule')
#     print(masked)
    if masked:
        subset = 'Masked' 
    else:
        subset = 'Unmasked'
    print(f'\multirow{{9}}{{*}}{{ {subset} }} & ')
    # print(fair_metrics)
    
    skip_head_sep = True
    for met in fair_metrics:
        result = []
        for mod in fair_metrics + ['unfair']:
            # print(met, mod)
            results = fair_results[met][mod]
            # print(results)
            mean = np.mean(results, 0)[0]
            se = np.std(results, 0)[0] / np.sqrt(len(results))
#             se = shorten_se(se)
#             se = f'{se:.3f}'
#             result.append(f"${mean:.3f} \\spm {se}$")
            result.append(f"{mean:.3f}")
        
        if skip_head_sep:
            resultrow = formal_fairness_name(met).replace('\\', '') + ' & ' + ' & '.join(result)
            skip_head_sep = False
        else:
            resultrow = formal_fairness_name(met).replace('\\', '').replace('SimFair', 'SF') + ' & ' + ' & '.join(result)
        resultrow = resultrow.replace('.0 ', ' ').replace('0.', '.')
        print(resultrow + '\\\\')

    print('\cmidrule(l){1-1}\cmidrule(l){2-8}')
    for perform_metric in list(perform_results.keys()):
        if 'F1' in perform_metric:

            result = []
            for mod in fair_metrics + ['unfair']:
                results = perform_results[perform_metric][mod]
                mean = np.mean(results, 0)[0]
                se = np.std(results, 0)[0] / np.sqrt(len(results))
                se = f'{se:.3f}'
                result.append(f"{mean:.3f}")
            resultrow = formal_metric_name(perform_metric).replace('\\', '').replace('SimFair', 'SF') + ' & ' + ' & '.join(result)
            resultrow = resultrow.replace('.0 ', ' ').replace('0.', '.')
            print(resultrow + '\\\\')
    print('\\bottomrule')

In [4]:
show_valid_replication('adult', 'l2', '0', fair_coeff=1)
print('\n' * 5)
# show_valid_replication('adult', 0, True)

 & w/ DP reg & w/ $ s_{ 0.01 } $-SF reg & w/ $ s_{ 1.0 } $-SF reg & w/ $ s_{ 5.0 } $-SF reg & w/ $ s_{ 10.0 } $-SF reg & w/ EOp reg & w/ Ele-EOp reg & w/o Reg\\
\midrule
\multirow{9}{*}{ Unmasked } & 
DP & .144 & .149 & .149 & .157 & .163 & .160 & .157 & .171\\
$ s_{ .01 } $-SF & .144 & .149 & .150 & .158 & .163 & .159 & .157 & .171\\
$ s_{ 1 } $-SF & .145 & .150 & .150 & .159 & .164 & .160 & .158 & .172\\
$ s_{ 5 } $-SF & .146 & .152 & .152 & .158 & .163 & .162 & .160 & .174\\
$ s_{ 10 } $-SF & .145 & .152 & .152 & .157 & .163 & .160 & .159 & .173\\
EOp & .146 & .152 & .152 & .157 & .162 & .160 & .159 & .173\\
Ele-EOp & .171 & .177 & .173 & .181 & .183 & .175 & .164 & .178\\
\cmidrule(l){1-1}\cmidrule(l){2-8}
instance-F1 & .598 & .595 & .596 & .595 & .601 & .598 & .592 & .593\\
micro-F1 & .578 & .575 & .577 & .574 & .581 & .576 & .566 & .569\\
macro-F1 & .218 & .211 & .209 & .214 & .203 & .213 & .215 & .212\\
\bottomrule








In [5]:
show_valid_replication('adult', 'l2', '0', fair_coeff=0.1)
print('\n' * 5)
# show_valid_replication('adult', 0, True)

 & w/ DP reg & w/ $ s_{ 0.01 } $-SF reg & w/ $ s_{ 1.0 } $-SF reg & w/ $ s_{ 5.0 } $-SF reg & w/ $ s_{ 10.0 } $-SF reg & w/ EOp reg & w/ Ele-EOp reg & w/o Reg\\
\midrule
\multirow{9}{*}{ Unmasked } & 
DP & .196 & .199 & .201 & .207 & .203 & .216 & .224 & .223\\
$ s_{ .01 } $-SF & .197 & .199 & .201 & .207 & .202 & .217 & .223 & .222\\
$ s_{ 1 } $-SF & .197 & .199 & .200 & .207 & .203 & .217 & .223 & .222\\
$ s_{ 5 } $-SF & .211 & .215 & .216 & .222 & .216 & .234 & .237 & .236\\
$ s_{ 10 } $-SF & .252 & .253 & .255 & .262 & .255 & .273 & .278 & .284\\
EOp & .253 & .259 & .263 & .271 & .262 & .279 & .282 & .287\\
Ele-EOp & .501 & .508 & .512 & .514 & .510 & .511 & .496 & .521\\
\cmidrule(l){1-1}\cmidrule(l){2-8}
instance-F1 & .597 & .599 & .600 & .598 & .599 & .592 & .590 & .589\\
micro-F1 & .582 & .581 & .581 & .578 & .580 & .568 & .565 & .562\\
macro-F1 & .213 & .214 & .217 & .221 & .213 & .207 & .215 & .219\\
\bottomrule








In [6]:
show_valid_replication('credit', 'l2', '0', fair_coeff=1)
print('\n' * 5)
# show_valid_replication('adult', 0, True)

 & w/ DP reg & w/ $ s_{ 0.01 } $-SF reg & w/ $ s_{ 1.0 } $-SF reg & w/ $ s_{ 5.0 } $-SF reg & w/ $ s_{ 10.0 } $-SF reg & w/ EOp reg & w/ Ele-EOp reg & w/o Reg\\
\midrule
\multirow{9}{*}{ Unmasked } & 
DP & .311 & .307 & .322 & .295 & .312 & .426 & .191 & .476\\
$ s_{ .01 } $-SF & .312 & .309 & .319 & .293 & .314 & .437 & .190 & .483\\
$ s_{ 1 } $-SF & .312 & .308 & .322 & .302 & .312 & .428 & .192 & .484\\
$ s_{ 5 } $-SF & .391 & .388 & .407 & .381 & .380 & .512 & .239 & .592\\
$ s_{ 10 } $-SF & .471 & .464 & .476 & .436 & .432 & .606 & .293 & .687\\
EOp & .395 & .389 & .461 & .397 & .396 & .609 & .219 & .664\\
Ele-EOp & .415 & .408 & .428 & .437 & .441 & .477 & .161 & .497\\
\cmidrule(l){1-1}\cmidrule(l){2-8}
macro-F1 & .352 & .336 & .340 & .341 & .332 & .352 & .318 & .340\\
micro-F1 & .561 & .553 & .552 & .570 & .559 & .539 & .552 & .539\\
\bottomrule








In [5]:
show_valid_replication('simulation', 'l2', '0', fair_coeff=1)
print('\n' * 5)
# show_valid_replication('adult', 0, True)

 & w/ DP reg & w/ $ s_{ 0.01 } $-SF reg & w/ $ s_{ 1.0 } $-SF reg & w/ $ s_{ 5.0 } $-SF reg & w/ $ s_{ 10.0 } $-SF reg & w/ EOp reg & w/ Ele-EOp reg & w/o Reg\\
\midrule
\multirow{9}{*}{ Unmasked } & 
DP & .043 & .042 & .039 & .043 & .040 & .054 & .042 & .061\\
$ s_{ .01 } $-SF & .043 & .042 & .040 & .042 & .041 & .052 & .041 & .062\\
$ s_{ 1 } $-SF & .044 & .042 & .040 & .041 & .041 & .053 & .042 & .063\\
$ s_{ 5 } $-SF & .045 & .045 & .044 & .045 & .041 & .058 & .044 & .066\\
$ s_{ 10 } $-SF & .051 & .048 & .045 & .046 & .046 & .064 & .050 & .090\\
EOp & .062 & .053 & .055 & .056 & .054 & .070 & .054 & .080\\
Ele-EOp & .010 & .009 & .009 & .009 & .009 & .011 & .009 & .008\\
\cmidrule(l){1-1}\cmidrule(l){2-8}
instance-F1 & .647 & .646 & .649 & .059 & .647 & .648 & .603 & .085\\
micro-F1 & .659 & .658 & .661 & .084 & .659 & .660 & .631 & .127\\
macro-F1 & .655 & .654 & .657 & .090 & .655 & .656 & .628 & .130\\
\bottomrule






